<a href="https://colab.research.google.com/github/selva-mani-007/gen-ai/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install Wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for Wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=392dd1b70e2baa85e5dc18259603f19c8b2f3eecadd64aea9e75582f799b91a5
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built Wikipedia


In [ ]:
!pip install wikipedia-api


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=b88410a391e2c07bd3773e7bd0ab19ef40baae3ee519de275669cd11c0ec6dbf
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.3 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass
from langchain_community.document_loaders import WikipediaLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI  # Using OpenRouter
from langchain.chains import RetrievalQA

# Set OpenRouter API Key securely
os.environ["OPENROUTER_API_KEY"] = getpass("Enter your OpenRouter API Key: ")

# Check API Key
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OpenRouter API Key is missing. Set it as an environment variable.")

# Initialize Wikipedia Loader
def load_wikipedia_data(query):
    loader = WikipediaLoader(query=query, lang='en')
    return loader.load()

# Initialize embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load Wikipedia data and create FAISS index
def build_faiss_index(topic):
    documents = load_wikipedia_data(topic)
    if not documents:
        raise ValueError(f"No Wikipedia data found for {topic}")

    texts = [doc.page_content for doc in documents]
    vector_db = FAISS.from_texts(texts, embedding_model)
    return vector_db

# Load OpenRouter GPT model
llm = ChatOpenAI(
    model_name="openai/gpt-3.5-turbo",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

# Create Retrieval QA Chain
def create_rag_chain(vector_db):
    retriever = vector_db.as_retriever(search_kwargs={"k": 3})  # Fetch top 3 relevant docs
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

if __name__ == '__main__':
    topic = input("Enter a topic to fetch Wikipedia data: ").strip()
    if not topic:
        print("Error: Topic cannot be empty.")
        exit()

    try:
        vector_db = build_faiss_index(topic)
        qa_chain = create_rag_chain(vector_db)

        while True:
            user_query = input(f"Ask a question about {topic} (or type 'exit' to quit): ").strip()
            if user_query.lower() == 'exit':
                print("Goodbye!")
                break
            response = qa_chain.run(user_query)
            print("\nResponse:", response)
    except Exception as e:
        print("Error:", str(e))


Enter your OpenRouter API Key: ··········


<ipython-input-1-6cf0ac649daf>:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

Enter a topic to fetch Wikipedia data: virat kohli
Ask a question about virat kohli (or type 'exit' to quit): wife name


<ipython-input-1-6cf0ac649daf>:62: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(user_query)



Response: Virat Kohli's wife is actress Anushka Sharma.
Ask a question about virat kohli (or type 'exit' to quit): no of international hundreds

Response: I don't have specific information about the number of international hundreds scored by Rohit Sharma.
Ask a question about virat kohli (or type 'exit' to quit): ipl franchise name

Response: The franchise names mentioned in the context are Kolkata Knight Riders (KKR) and Royal Challengers Bengaluru (RCB).
Ask a question about virat kohli (or type 'exit' to quit): exit
Goodbye!


In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 35.4 MB/s eta 0:00:00


In [ ]:

import os
import fitz  # PyMuPDF for PDF processing
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI  # Using OpenRouter
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set OpenRouter API Key (ensure it's set in your environment)
OPENROUTER_API_KEY = "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"

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

# Load PDF and create FAISS index
def build_faiss_index(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_text(text)

    documents = [Document(page_content=t) for t in texts]

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = FAISS.from_documents(documents, embedding_model)
    return vector_db

# Load OpenRouter GPT model
llm = ChatOpenAI(
    model_name="openai/gpt-3.5-turbo",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

# Create Retrieval QA Chain
def create_rag_chain(vector_db):
    retriever = vector_db.as_retriever()
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

if __name__ == '__main__':
    pdf_path = input("Enter the path to your PDF file: ")
    vector_db = build_faiss_index(pdf_path)
    qa_chain = create_rag_chain(vector_db)

    while True:
        user_query = input("Ask a question from the PDF (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = qa_chain.run(user_query)
        print("Response:", response)

Enter the path to your PDF file: /content/Ch.01_Introduction_ to_computers.pdf
Ask a question from the PDF (or type 'exit' to quit): what is computer
Response: A computer is an electronic device that operates under the control of instructions stored in its memory. It can accept data as input, process the data according to specified rules, produce information as output, and store information for future use. Computers consist of hardware (physical components like monitor, keyboard, storage devices) and software (programs and instructions). They can be classified based on size and power into categories like personal computers, workstations, minicomputers, mainframes, and supercomputers. Computers are versatile and can perform various tasks but can only do what they have been programmed to do.
Ask a question from the PDF (or type 'exit' to quit): exit


In [ ]:

import requests
import fitz  # PyMuPDF for PDF handling (if needed)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI  # LLaMA/Mistral via OpenRouter
from langchain.chains import RetrievalQA
from langchain.schema import Document

# PubMed API Constants
PUBMED_API_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
PUBMED_FETCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# Initialize embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# OpenRouter API Key for LLaMA/Mistral
OPENROUTER_API_KEY = "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"

# Initialize LLM (LLaMA/Mistral)
llm = ChatOpenAI(
    model_name="mistralai/mistral-7b-instruct",  # You can change to "meta-llama/llama-2-7b-chat"
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

def fetch_pubmed_articles(query, max_results=5):
    """ Fetch PubMed article abstracts based on query. """
    params = {
        "db": "pubmed",
        "term": query,
        "retmode": "json",
        "retmax": max_results
    }
    response = requests.get(PUBMED_API_URL, params=params)
    article_ids = response.json().get("esearchresult", {}).get("idlist", [])

    if not article_ids:
        return []

    # Fetch article details
    fetch_params = {
        "db": "pubmed",
        "id": ",".join(article_ids),
        "retmode": "text",
        "rettype": "abstract"
    }
    response = requests.get(PUBMED_FETCH_URL, params=fetch_params)
    return response.text.split("\n\n")  # Split articles

def build_faiss_index(articles):
    """ Process articles into FAISS vector store. """
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = [chunk for article in articles for chunk in text_splitter.split_text(article)]

    documents = [Document(page_content=chunk) for chunk in text_chunks]
    vector_db = FAISS.from_documents(documents, embedding_model)
    return vector_db

def create_rag_chain(vector_db):
    """ Create Retrieval-Augmented Generation (RAG) Chain. """
    retriever = vector_db.as_retriever()
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

if __name__ == '__main__':
    user_query = input("Enter a medical topic (e.g., 'Diabetes treatment', 'COVID-19 vaccines'): ")
    print(f"Fetching articles related to: {user_query}")

    articles = fetch_pubmed_articles(user_query)

    if not articles:
        print("No relevant articles found on PubMed.")
    else:
        print("Building vector database...")
        vector_db = build_faiss_index(articles)
        qa_chain = create_rag_chain(vector_db)

        print("System ready! Ask health-related questions.")
        while True:
            question = input("Ask a health question (or type 'exit' to quit): ")
            if question.lower() == 'exit':
                break
            response = qa_chain.run(question)
            print("Response:", response)

Enter a medical topic (e.g., 'Diabetes treatment', 'COVID-19 vaccines'): insomnia
Fetching articles related to: insomnia
Building vector database...
System ready! Ask health-related questions.
Ask a health question (or type 'exit' to quit): how to prevent it
Response:  The provided context does not provide information on how to prevent a specific disease or condition. It simply lists the authors, their affiliations, and the publication details of two different research articles. To find information on how to prevent a disease or condition, you would need to consult the research articles themselves or other authoritative sources such as the Centers for Disease Control and Prevention (CDC) or the World Health Organization (WHO).
Ask a health question (or type 'exit' to quit): then why you are here get lost
Response:  Based on the provided context, it appears that the titles are for academic articles related to the psychological status of pregnant women in Nairobi County, Kenya during the

In [ ]:
!pip install openai-whisper


In [ ]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-0birup1_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-0birup1_
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=ac959db59f5d0850e0ece8597c1f39643cb71a9cfd3de87be6377678dda5c441
  Stored in directory: /tmp/pip-ephem-wheel-cache-ycx_yzcm/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [2]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 44.4 MB/s eta 0:00:00


In [4]:
!pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s

In [6]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.7 MB/s eta 0:00:00


In [8]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [10]:

import os
import torch
import fitz  # PyMuPDF (for PDF handling)
import whisper  # OpenAI Whisper (for speech recognition)
import faiss  # Facebook AI Similarity Search
import numpy as np
from PIL import Image
from google.colab import files  # For file uploads in Colab
from transformers import CLIPProcessor, CLIPModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Set OpenRouter API Key (Replace with your actual key)
OPENROUTER_API_KEY ="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"
# Load LLM (Mistral or LLaMA via OpenRouter)
llm = ChatOpenAI(
    model_name="mistralai/mistral-7b-instruct",
    temperature=0.7,
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

# Load Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load CLIP Model for Image Processing
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load Whisper Model for Audio Processing
whisper_model = whisper.load_model("base")

# === FILE UPLOAD FUNCTIONS ===
def upload_file(file_type):
    """Upload a file of a specific type (PDF, image, or audio)."""
    print(f"Please upload your {file_type} file (or skip if not required)...")
    uploaded = files.upload()
    return list(uploaded.keys())[0] if uploaded else None  # Return filename if uploaded, else None

# === TEXT PROCESSING ===
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    if not pdf_path:
        return None
    doc = fitz.open(pdf_path)
    return "\n".join(page.get_text("text") for page in doc)

def build_text_faiss(text_data):
    """Create a FAISS index for text-based retrieval."""
    if not text_data:
        return None
    text_splitter = text_data.split("\n")  # Split text into sentences
    text_docs = [Document(page_content=chunk) for chunk in text_splitter if chunk.strip()]
    vector_db = FAISS.from_documents(text_docs, embedding_model)
    return vector_db

# === IMAGE PROCESSING ===
def encode_image(image_path):
    """Encode an image using CLIP."""
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features.cpu().numpy()

def build_image_faiss(image_paths):
    """Create a FAISS index for image-based retrieval."""
    if not image_paths:
        return None, {}

    d = 512  # CLIP feature dimension
    index = faiss.IndexFlatL2(d)
    image_store = {}

    for img_path in image_paths:
        img_vector = encode_image(img_path)
        index.add(img_vector)
        image_store[len(image_store)] = img_path  # Store mapping

    return index, image_store

# === AUDIO PROCESSING ===
def transcribe_audio(audio_path):
    """Transcribe audio to text using Whisper."""
    if not audio_path:
        return None
    result = whisper_model.transcribe(audio_path)
    return result["text"]

def build_audio_faiss(audio_texts):
    """Create FAISS index for transcribed audio data."""
    if not audio_texts:
        return None
    audio_docs = [Document(page_content=txt) for txt in audio_texts]
    vector_db = FAISS.from_documents(audio_docs, embedding_model)
    return vector_db

# === MULTIMODAL QUERY HANDLING ===
def multimodal_query(query, text_db, image_index, image_store, audio_db):
    """Retrieve relevant results from text, image, and audio sources."""

    # Text-based retrieval
    text_result = text_db.similarity_search(query, k=1) if text_db else None

    # Image-based retrieval
    image_result = None
    if image_index is not None and os.path.exists(query):  # If query is an image file
        query_embedding = encode_image(query)
        distances, indices = image_index.search(query_embedding, k=1)
        image_result = image_store.get(indices[0][0], None)

    # Audio-based retrieval
    audio_result = audio_db.similarity_search(query, k=1) if audio_db else None

    return {
        "text": text_result[0].page_content if text_result else "No relevant text found.",
        "image": image_result if image_result else "No matching image found.",
        "audio": audio_result[0].page_content if audio_result else "No relevant audio found."
    }

# === MAIN EXECUTION ===
if __name__ == "__main__":
    # Upload files via Colab (optional uploads)
    pdf_path = upload_file("PDF")
    image_paths = [upload_file("image")] if input("Upload an image? (y/n): ").lower() == "y" else None
    audio_path = upload_file("audio") if input("Upload an audio file? (y/n): ").lower() == "y" else None

    print("\n🔄 Processing PDF...")
    text_faiss_db = build_text_faiss(extract_text_from_pdf(pdf_path)) if pdf_path else None

    print("🔄 Processing Images...")
    image_faiss_index, image_store = build_image_faiss(image_paths) if image_paths else (None, {})

    print("🔄 Processing Audio...")
    transcribed_text = transcribe_audio(audio_path) if audio_path else None
    audio_faiss_db = build_audio_faiss([transcribed_text]) if transcribed_text else None

    print("\n✅ System Ready! Ask multimodal queries...")

    while True:
        user_query = input("\nEnter a text query (or provide an image/audio file path, type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break

        results = multimodal_query(user_query, text_faiss_db, image_faiss_index, image_store, audio_faiss_db)

        print("\n🔹 **Results:**")
        print(f"📖 **Text Answer:** {results['text']}")
        print(f"🖼️ **Image Match:** {results['image']}")
        print(f"🔊 **Audio Answer:** {results['audio']}")


Please upload your PDF file (or skip if not required)...


Saving Ch.01_Introduction_ to_computers.pdf to Ch.01_Introduction_ to_computers.pdf
Upload an image? (y/n): n
Upload an audio file? (y/n): 

🔄 Processing PDF...
🔄 Processing Images...
🔄 Processing Audio...

✅ System Ready! Ask multimodal queries...

Enter a text query (or provide an image/audio file path, type 'exit' to quit): what is a computer

🔹 **Results:**
📖 **Text Answer:** Computer 
🖼️ **Image Match:** No matching image found.
🔊 **Audio Answer:** No relevant audio found.

Enter a text query (or provide an image/audio file path, type 'exit' to quit): exit
